## Imports, Connect to Database

In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
#'/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

Connecting kui@ucsd-demo-db.datajoint.io:3306


./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD589'
yaml_file = 'shape_params-aws.yaml'
scripts_dir = os.environ['REPO_DIR']

## Create Process Table
--script  shapeolopy_code/scripts/Cell_generator.py

In [25]:
Process.drop()

`kui_diffusionmap`.`__process` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [26]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    size_positive_properties : int   #size of file
    size_negative_properties : int   #size of file
    """
#     size_positive_images     : int   #size of file
#     size_negative_images     : int   #size of file
    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            #img_name = state+'_img_fp'
            key_item = 'size_'+state+'_properties'
            #key_img = 'size_'+state+'_images'
            s3_fp = (Structure & key).fetch1(item_name)
            #img_fp = (Structure & key).fetch1(img_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000)
                #report2 = self.client.stat_object(self.bucket, img_fp)
                #key[key_img] = int(report2.size/1000000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                #upload_img_fp = 's3://'+self.bucket+'/'+img_fp
                pkl_out_file = os.environ['ROOT_DIR']+s3_fp
                #img_out_file = os.environ['ROOT_DIR']+img_fp
                ! aws s3 cp $pkl_out_file $upload_fp
                #! aws s3 cp $img_out_file $upload_img_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000)
                #report2 = self.client.stat_object(self.bucket, img_fp)
                #key[key_img] = int(report2.size/1000000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [27]:
diffusion = Process()

In [28]:
diffusion.populate()#reserve_jobs=True

populating for  5N
populating for  6N
populating for  7N
populating for  7n
populating for  Amb
populating for  LC
populating for  LRt
populating for  Pn
populating for  Tz
populating for  VLL
populating for  RMC
populating for  SNC
populating for  SNR
populating for  3N
populating for  4N
populating for  Sp5I
populating for  Sp5O
populating for  Sp5C
populating for  PBG
populating for  10N
populating for  VCA
populating for  VCP
populating for  DC
populating for  AP
populating for  12N
populating for  RtTg
populating for  SC
populating for  IC


In [29]:
Process.fetch(as_dict=True)

[OrderedDict([('id', 0),
              ('size_positive_properties', 67),
              ('size_negative_properties', 67)]),
 OrderedDict([('id', 1),
              ('size_positive_properties', 7),
              ('size_negative_properties', 7)]),
 OrderedDict([('id', 2),
              ('size_positive_properties', 136),
              ('size_negative_properties', 136)]),
 OrderedDict([('id', 3),
              ('size_positive_properties', 193),
              ('size_negative_properties', 193)]),
 OrderedDict([('id', 4),
              ('size_positive_properties', 19),
              ('size_negative_properties', 19)]),
 OrderedDict([('id', 5),
              ('size_positive_properties', 36),
              ('size_negative_properties', 37)]),
 OrderedDict([('id', 6),
              ('size_positive_properties', 116),
              ('size_negative_properties', 116)]),
 OrderedDict([('id', 7),
              ('size_positive_properties', 155),
              ('size_negative_properties', 155)]),
 OrderedDi